In [3]:
import pandas as pd
import numpy as np
import numpy.linalg as la
import statsmodels.discrete.discrete_model as sm
from statsmodels.tools.tools import add_constant
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from linearmodels import PanelOLS, RandomEffects
from pyeconometrics.panel_discrete_models import FixedEffectPanelModel
from scipy import stats
from statsmodels.tsa.stattools import adfuller
import hausman

warnings.filterwarnings('ignore')
plt.style.use('seaborn')
pd.options.mode.chained_assignment = None

In [5]:
# read data
CPI = pd.read_csv('CPI.csv')
GDP = pd.read_csv('GDP.csv')
dta = pd.read_csv('final_exr.csv')

# merge CPI data
CPI = pd.melt(CPI, id_vars=["Country Name", "Country Code"], var_name="Year", value_name="CPI").sort_values(by=["Country Name", "Country Code"])
CPI.columns = ['originn', 'country_code', 'year', 'CPI']
CPI.year = CPI.year.astype(int)
dta = pd.merge(dta, CPI, how='left', on=['year','originn'])

# merge CPI of China data
CPIbase = CPI[CPI.originn == u'中国']
CPIbase = CPIbase.drop(['originn', 'country_code'], axis=1)
CPIbase.columns = ['year', 'CPIbase']
dta = pd.merge(dta, CPIbase, how='left', on=['year'])

# merge GDP data
GDP = pd.melt(GDP, id_vars=["Country Code"], var_name="Year", value_name="GDP").sort_values(by=["Country Code","Year"])
GDP.columns = ['country_code', 'year', 'GDP']
GDP.year = GDP.year.astype(int)
dta = pd.merge(dta, GDP, how='left', on=['country_code','year'])

In [6]:
# drop unused variables
dta = dta.drop(['id', 'name', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1)

# define new variables
dta['export_delivery_rate'] = dta.export / dta.total_sale
dta['D'] = pd.cut(dta['export_delivery_rate'], bins=[0.0,0.2,0.5,1.0], labels=[1.0,2.0,3.0], include_lowest=True)
dta['D2'] = 0
dta['D2'][dta['D'] == 2.0] = 1
dta['D3'] = 0
dta['D3'][dta['D'] == 3.0] = 1

# drop NaN
dta = dta[(dta.export_delivery_rate.notnull()) & (dta.CPI.notnull()) & (dta.GDP.notnull()) \
          & (dta.aemployment != 0) & (dta.exchange_rate.notnull())]

# define new variables
dta['Quantity'] = dta.ex * dta.quantity_ex + dta.im * dta.quantity_im
dta['processing_trade'] = 0
dta['processing_trade'][(dta.regime_name == u'进料加工贸易') | 
                        (dta.regime_name == u'来料加工装配贸易') | 
                        (dta.regime_name == u'出料加工贸易')] = 1.0

dta['mainbusiness_profit'] = dta.mainbusiness_revenue - dta.mainbusiness_cost

dta['debt_asset_ratio'] = (dta.total_asset - dta.owner_rights) / dta.total_asset

In [7]:
# filter, drop illegal variables
temp = dta[(dta.owner_rights>0) #& (dta.mainbusiness_profit>0) & (dta.total_profit>0)
           & (dta.totaloutput>0) & (dta.wage_payable>0) #& (dta.Quantity>0)  
           & (dta.debt_asset_ratio>0) & (dta.debt_asset_ratio<1)
           & (dta.export_delivery_rate>0) & (dta.export_delivery_rate<=1) & (dta.price>0) & (dta.mainbusiness_revenue!=0) 
           & (dta.mainbusiness_cost!=0) & ((dta.value_ex>0) | (dta.value_im>0)) 
           & (dta.valueadded>0) & (dta.total_asset>0) 
           & (dta.aemployment>=8) & (dta.setup_year>=1949.0)
           & (dta.total_asset>dta.totalfixed_asset) 
           & (dta.totalfixed_asset>0) & (dta.totalintermediate_input>0)]  

In [8]:
# Import Only
len(temp[(temp.ex==1) & (temp.im==0)]) / len(temp)

0.6152392096854044

In [9]:
# Export Only
len(temp[(temp.ex==0) & (temp.im==1)]) / len(temp)

0.37416425381680346

In [10]:
# Neither
len(temp[(temp.ex==0) & (temp.im==0)]) / len(temp)

0.0

In [11]:
# Both
len(temp[(temp.ex==1) & (temp.im==1)]) / len(temp)

0.01059653649779218

In [12]:
len(temp)

199971

In [ ]:
# set index: firm name and year
temp = temp.set_index(['frdwdm','year'])

# define new variables
temp['profit_rate'] = temp.mainbusiness_profit / temp.mainbusiness_revenue

temp['trade_depend_ex'] = (temp.value_ex * temp.ex) / temp.mainbusiness_revenue
temp['trade_depend_im'] = (temp.value_im * temp.im) / temp.mainbusiness_cost
temp['exchange_rate_depend'] = temp.exchange_rate * temp.trade_depend_ex + temp.exchange_rate * temp.trade_depend_im
temp['exchange_rate_CPI_adj'] = temp.CPIbase / (temp.CPI * temp.exchange_rate_depend)

temp['GDP_depend'] = temp.GDP #* temp.trade_depend_ex + temp.GDP * temp.trade_depend_im

temp['processing_trade_exr'] = np.log(temp.exchange_rate_CPI_adj) * temp.processing_trade
temp['tfp'] = np.log(temp.totaloutput/temp.aemployment) - 1/3 * np.log(temp.total_asset/temp.aemployment)

temp['exchange_rate_medium'] = temp.D2 * np.log(temp.exchange_rate_CPI_adj)
temp['exchange_rate_high'] = temp.D3 * np.log(temp.exchange_rate_CPI_adj)

In [ ]:
# Fixed Effect Model
temp_y1 = np.log(temp['total_profit'])
temp_y4 = np.log(temp['mainbusiness_profit']) 
temp_y5 = np.log(temp['profit_rate'])
temp_y6 = np.log(temp['export_delivery_rate'])

temp_X1 = temp[['wage_payable','valueadded','debt_asset_ratio',
                'Quantity','price','GDP_depend','exchange_rate_CPI_adj']].apply(np.log)
temp_X1 = temp_X1.merge(temp.processing_trade_exr.to_frame(), left_index=True, right_index=True)
temp_X1 = temp_X1.merge(temp.tfp.to_frame(), left_index=True, right_index=True)
temp_X1 = temp_X1.merge(temp.exchange_rate_medium.to_frame(), left_index=True, right_index=True)
temp_X1 = temp_X1.merge(temp.exchange_rate_high.to_frame(), left_index=True, right_index=True)

dta_temp1 = pd.concat([temp_y1, temp_X1], axis=1) # total_profit
dta_temp4 = pd.concat([temp_y4, temp_X1], axis=1) # mainbusiness_profit
dta_temp5 = pd.concat([temp_y5, temp_X1], axis=1) # profit_rate

In [ ]:
# Summary Statistics
dta_temp1.describe()

In [ ]:
# Fixed Effect Regression on Total Profit 
mod11 = PanelOLS.from_formula('total_profit ~ 1 + wage_payable + valueadded + \
                              Quantity + price + tfp + debt_asset_ratio + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium + exchange_rate_high + \
                              EntityEffects + TimeEffects', dta_temp1)
res11 = mod11.fit(cov_type='heteroskedastic')
print(res11) 

In [ ]:
# Random Effect Regression on Total Profit 
random_mod11 = RandomEffects.from_formula('total_profit ~ 1 + wage_payable + valueadded + \
                              Quantity + price + tfp + debt_asset_ratio + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium + exchange_rate_high + \
                              EntityEffects + TimeEffects', dta_temp1)
random_res11 = random_mod11.fit(cov_type='heteroskedastic')
print(random_res11)

In [ ]:
# Hausman Test
hausman(res11, random_res11)

In [ ]:
# regression on mainbusiness_revenue is bad
# Fixed Effect Regression on Main Business Profit
mod12 = PanelOLS.from_formula('mainbusiness_profit ~ 1 + wage_payable + valueadded + \
                              Quantity + price + tfp + debt_asset_ratio + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium + exchange_rate_high + \
                              EntityEffects + TimeEffects', dta_temp4)
res12 = mod12.fit(cov_type='heteroskedastic')
print(res12) 

In [ ]:
# Random Effect Regression on Total Profit 
random_mod12 = RandomEffects.from_formula('mainbusiness_profit ~ 1 + wage_payable + valueadded + \
                              Quantity + price + tfp + debt_asset_ratio + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium + exchange_rate_high + \
                              EntityEffects + TimeEffects', dta_temp4)
random_res12 = random_mod12.fit(cov_type='heteroskedastic')
print(random_res12)

In [ ]:
# Hausman Test
hausman(res12, random_res12)

In [ ]:
# Fixed Effect Regression on Profit Rate
mod13 = PanelOLS.from_formula('profit_rate ~ 1 + wage_payable + valueadded + \
                              Quantity + price + tfp + debt_asset_ratio + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium + exchange_rate_high + \
                              EntityEffects + TimeEffects', dta_temp5)
res13 = mod13.fit(cov_type='heteroskedastic')
print(res13) # mainbusiness_profit; profit_rate; mainbusiness_revenue

In [ ]:
# Random Effect Regression on Profit Rate
random_mod13 = RandomEffects.from_formula('profit_rate ~ 1 + wage_payable + valueadded + \
                              Quantity + price + tfp + debt_asset_ratio + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium + exchange_rate_high + \
                              EntityEffects + TimeEffects', dta_temp5)
random_res13 = random_mod13.fit(cov_type='heteroskedastic')
print(random_res13)

In [ ]:
# Hausman Test
hausman(res13, random_res13)

In [ ]:
# Prepare Data for Fixed Effect Regression on Export Quantity/Price
temp_X21 = temp[temp.ex == 1]
temp_X21 = temp_X21[temp_X21.quantity_ex>0]

temp_X21 = temp_X21[['wage_payable','wp_mainbusiness','valueadded','finance_expense', 
                'totalintermediate_input','fe_interest','totaloutput',
                'quantity_ex','value_ex','price','GDP_depend','exchange_rate_CPI_adj']].apply(np.log)
temp_X21 = temp_X21.merge(temp.processing_trade_exr.to_frame(), left_index=True, right_index=True)
temp_X21 = temp_X21.merge(temp.tfp.to_frame(), left_index=True, right_index=True)
temp_X21 = temp_X21.merge(temp.exchange_rate_medium.to_frame(), left_index=True, right_index=True)
temp_X21 = temp_X21.merge(temp.exchange_rate_high.to_frame(), left_index=True, right_index=True)

# Prepare Data for Fixed Effect Regression on Import Quantity/Price
temp_X22 = temp[temp.im == 1]
temp_X22 = temp_X22[temp_X22.quantity_im>0]

temp_X22 = temp_X22[['wage_payable','wp_mainbusiness','valueadded','finance_expense', 
                'totalintermediate_input','fe_interest','totaloutput',
                'quantity_im','price','GDP_depend','exchange_rate_CPI_adj']].apply(np.log)
temp_X22 = temp_X22.merge(temp.processing_trade_exr.to_frame(), left_index=True, right_index=True)
temp_X22 = temp_X22.merge(temp.tfp.to_frame(), left_index=True, right_index=True)
temp_X22 = temp_X22.merge(temp.exchange_rate_medium.to_frame(), left_index=True, right_index=True)
temp_X22 = temp_X22.merge(temp.exchange_rate_high.to_frame(), left_index=True, right_index=True)

In [ ]:
# Fixed Effect Regression on Export Quantity
mod21 = PanelOLS.from_formula('quantity_ex ~ 1 + wage_payable + \
                              valueadded + price + tfp + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium + exchange_rate_high + \
                              EntityEffects + TimeEffects', temp_X21)
res21 = mod21.fit(cov_type='heteroskedastic')
print(res21)

In [ ]:
# Random Effect Regression on Export Quantity
random_mod21 = RandomEffects.from_formula('quantity_ex ~ 1 + wage_payable + \
                              valueadded + price + tfp + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium + exchange_rate_high + \
                              EntityEffects + TimeEffects', temp_X21)
random_res21 = random_mod21.fit(cov_type='heteroskedastic')
print(random_res21)

In [ ]:
# Hausman Test
hausman(res21, random_res21)

In [ ]:
# Fixed Effect Regression on Import Quantity
mod22 = PanelOLS.from_formula('quantity_im ~ 1 + wage_payable + \
                              valueadded + price + tfp + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium + exchange_rate_high + \
                              EntityEffects + TimeEffects', temp_X22)
res22 = mod22.fit(cov_type='heteroskedastic')
print(res22)

In [ ]:
# Random Effect Regression on Import Quantity
random_mod22 = RandomEffects.from_formula('quantity_im ~ 1 + wage_payable + \
                              valueadded + price + tfp + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium + exchange_rate_high + \
                              EntityEffects + TimeEffects', temp_X22)
random_res22 = random_mod22.fit(cov_type='heteroskedastic')
print(random_res22)

In [ ]:
# Hausman Test
hausman(res22, random_res22)

In [ ]:
# Fixed Effect Regression on Export Price
mod31 = PanelOLS.from_formula('price ~ 1 + wage_payable + \
                              valueadded + quantity_ex + tfp + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium + exchange_rate_high + \
                              EntityEffects + TimeEffects', temp_X21)
res31 = mod31.fit(cov_type='heteroskedastic')
print(res31)

In [ ]:
# Random Effect Regression on Export Price
random_mod31 = RandomEffects.from_formula('price ~ 1 + wage_payable + \
                              valueadded + quantity_ex + tfp + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium + exchange_rate_high + \
                              EntityEffects + TimeEffects', temp_X21)
random_res31 = random_mod31.fit(cov_type='heteroskedastic')
print(random_res31)

In [ ]:
# Hausman Test
hausman(res31, random_res31)

In [ ]:
# Fixed Effect Regression on Import Price
mod32 = PanelOLS.from_formula('price ~ 1 + wage_payable + \
                              valueadded + quantity_im + tfp + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium+exchange_rate_high + \
                              EntityEffects + TimeEffects', temp_X22)
res32 = mod32.fit(cov_type='heteroskedastic')
print(res32)

In [ ]:
# Random Effect Regression on Import Price
random_mod32 = RandomEffects.from_formula('price ~ 1 + wage_payable + \
                              valueadded + quantity_im + tfp + GDP_depend + \
                              exchange_rate_CPI_adj + processing_trade_exr + \
                              exchange_rate_medium+exchange_rate_high + \
                              EntityEffects + TimeEffects', temp_X22)
random_res32 = random_mod32.fit(cov_type='heteroskedastic')
print(random_res32)

In [ ]:
# Hausman Testd
hausman(res32, random_res32)

In [ ]:
# logit model on export
temp_y8 = temp['ex']
exog = add_constant(temp_X1)

logit1 = sm.Logit(temp_y8, exog)

result1 = logit1.fit()
print(result1.summary2())

In [ ]:
# logit model on import
temp_y9 = temp['im']
logit2 = sm.Logit(temp_y9, exog)

result2 = logit2.fit()
print(result2.summary2())

In [ ]:
temp.ownership_name.value_counts()

In [ ]:
temp.ownershipc.value_counts()

In [ ]:
temp.transport_name.value_counts()

In [ ]:
temp.columns

##### Variables Declaration

frdwdm(公司名); year(年份); youbian(邮编); setup_year(开业年); setup_month(开业月); 

totaloutput(工业总产值); newproduct_output(工业总产值-新产品产值); total_sale(工业销售产值); export(工业销售产值-出口交货值); aemployment(全部职工); valueadded(工业增加值); longrun_invest(长期投资); totalfixed_asset(固定资产合计); fixed_asset_op(固定资产-生产经营用); depreciate(累计折旧); depreciate_tyear(累计折旧-本年折旧); fixedasset_net(固定资产净值年平均余额); total_asset(资产总计); owner_rights(所有者权益合计); paicl_up_capital(实收资本); so_capital(国家资本金); co_capital(集体资本金); lp_capital(法人资本金); p_capital(个人资本金); gat_capital(港澳台资本金); foreign_capital(外商资本金); 

mainbusiness_revenue(主营业务收入); mainbusiness_cost(主营业务成本); finance_expense(财务费用); fe_interest(财务费用-利息支出); total_profit(利润总额); advertise_expense(广告费); wage_payable(本年应付工资总额); wp_mainbusiness(本年应付福利费总额); totalintermediate_input(工业中间投入合计); 


drop variables: id(公司代码); frdm(法人代码); fddbr(法人); quhao(区号); dianhua(固定电话); fenjihao(分机号); chuanzhen(传真); email; web; registertype(登记注册类型); lishu(隶属关系); totalasset(资产总计含缺省值); e_state(!=so_capital); e_collective(=co_capital); e_individual(=p_capital); e_legal_person(=lp_capital); e_hmt(=gat_capital); e_foreign(=foreign_capital); e_total(=paicl_up_capital); soe_sh, collect_sh, lp_sh, p_sh, hmt_sh, foreign_sh(shares); state, collective, private, hmt, foreign(dummies); frdmid; firmid; newid(=frdmid-1);

unclear: edu(职工教育费), rd_expense(研究开发费), code_cnty, cic_adj, BR_deflator, I, cic2(first two digits of cic_adj), lny, lnl, lnk, lnm, lnk_net, investment_net(投资收益), lninvest_net(=ln(investment_net)), lninvest, 'ownership', 'ownerships', 

##### Tips
增加虚拟变量和想要控制变量的交互项;
加上categorical variable：是否出口(两列回归);
企业所在省份及所在产业í该省份该产业占全国比重(分行业)í产业结构变化;
分所有制;
看文献：解释变量是如何构造的;
被解释变量：markup ratio((price - marginal cost)/price);
检验：自相关，异方差，共线性，内生性，稳健性 

背景，一句话
问题，理论+实证
发现123

产品结构

TFP内生性

行业

cluster error多一个

TFP->dummy

D->25以内 75以上

进口、出口的比值，dummy
所有制

行业出口结构解决内生性问题

##### group by
贸易方式regime or regime_name, 地理位置youbian, 行业icode, 城市code_city or code_cityn, 运输方式transport or transport_name, 控股konggu, 所有制ownershipc, profit_rate利润率

In [ ]:
# visualzation for Total Number of Firms in Processing Trade Industry, Total Output, Export Share, Import Share
df1 = temp[temp.ex == 1]
df2 = temp[temp.im == 1]

total_output = temp.groupby(['year'])['totaloutput'].sum()
total_output1 = df1.groupby(['year'])['totaloutput'].sum()
total_output2 = df2.groupby(['year'])['totaloutput'].sum()
total_export = df1.groupby(['year'])['value_ex'].sum() / (total_output1*10)
total_import = df2.groupby(['year'])['value_im'].sum() / total_output2

df = temp[temp.processing_trade==1]
total = temp.groupby(['year']).size()
ts = df.year.value_counts().sort_index() 

del total_output.index.name
del total_export.index.name
del total_import.index.name

fig = plt.figure(figsize=(18,8))

ax1 = plt.subplot(2, 2, 1)
ts.plot(color='blue', grid=True, label='Number of Firms in Processing Trade')
ax1.legend(loc="upper left")

ax2 = plt.subplot(2, 2, 2)
total_output.plot(color='blue', grid=True, label='Total Output')
ax2.legend(loc="upper left")

ax3 = plt.subplot(2, 2, 3)
total_export.plot(color='blue', grid=True, label='Export Share')
ax3.legend(loc="upper right")

ax4 = plt.subplot(2, 2, 4)
total_import.plot(color='blue', grid=True, label='Import Share')
ax4.legend(loc="upper right")

plt.show()
fig.savefig('plot.png')

In [ ]:
# single plot: plot the number of firms in processing trade industry

fig = plt.figure(figsize=(12,5))

ax1 = ts.plot(color='blue', grid=True, label='Number')

h1, l1 = ax1.get_legend_handles_labels()

plt.legend(h1, l1, loc=3, prop={'size': 17})
plt.show()
fig.savefig('plot1.png')

In [ ]:
# Fixed Effect logit model on export
temp_X4 = pd.concat([temp_y8, temp_X1], axis=1)
FE1 = FixedEffectPanelModel()
FE1.fit(temp_X4, 'ex', verbose=True)
FE1.summary()
# FE.beta
# FE.beta_se

# Fixed Effect logit model on import
temp_X5 = pd.concat([temp_y9, temp_X1], axis=1)
FE2 = FixedEffectPanelModel()
FE2.fit(temp_X5, 'im', verbose=True)
FE2.summary()